In [3]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

from PIL import Image
import skimage

import sys, os

from IPython.display import clear_output

# these magics ensure that external modules that are modified are also automatically reloaded
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
myPlanetKey = "664df19c4dcd42b3b7d14acd5b9936b4"

In [5]:
outputPath  = "/home/data/prop-39-school/imagery/"

try:
    os.makedirs(outputPath)
except OSError:
    pass

In [8]:
!ls

Planet Labs API Exploration.ipynb  planetlabs.py  planetlabs.pyc


# Testing Planet's Python client

In [18]:
from planet import api
client = api.ClientV1(api_key=myPlanetKey)

In [19]:
print "Total data available from Planet by constellation:"
for sceneType in ["landsat", "ortho"]:
    scenes = client.get_scenes_list(scene_type=sceneType)
    print "%7s: %d"%(sceneType, scenes.get()['count'])

Total data available from Planet by constellation:


AttributeError: 'ClientV1' object has no attribute 'get_scenes_list'

In [ ]:
client.get

In [ ]:
filters = {'cloud_cover.estimated.lt': 1}
# scenes = client.get_scenes_list(intersects=wkt, limit=None, **filters)
# scenes = client.get_scenes_list(scene_type='ortho')
# scenes = client.get_scenes_list(scene_type='landsat')
wkt = 'POINT(37.65 -122.09)'
scenes = client.get_scenes_list(scene_type='landsat', intersection=wkt, limit=None, **filters)
geojson= scenes.get()

print "%d scenes"%geojson['count']

In [ ]:
geojson['features'][0]

In [ ]:
scene_ids = [scene['id'] for scene in scenes.items_iter(5)]
print len(scene_ids)
print scene_ids

In [ ]:
metadata_body = client.get_scene_metadata(scene_id='LC81441102016256LGN00')
metadata = metadata_body.get()
print metadata['properties']['sat']

In [ ]:
callback = api.write_to_file(directory='/home/ubuntu/data/prop39/images-planet/')
callbacks = client.fetch_scene_geotiffs(scene_ids, callback=callback)
for cb in callbacks:
    body = cb.await()
    print 'downloaded', body.name, 'size=%s bytes' % len(body)

# Using raw Python

In [ ]:
# construct request for Planet API

# define AOI

# (longitude, latitude)
sf_nw = locations[0]
sf_se = locations[1]
sf_ne = (sf_se[0], sf_nw[1])
sf_sw = (sf_nw[0], sf_se[1])

# Using WKT

from shapely.geometry import Polygon
from shapely.wkt import dumps as wkt_dumps

poly = Polygon([sf_nw, sf_ne, sf_se, sf_sw, sf_nw])
intersects = wkt_dumps(poly)

# Or, using GeoJSON

import geojson

poly = geojson.Polygon([[sf_nw, sf_ne, sf_se, sf_sw, sf_nw]])
intersects = geojson.dumps(poly)

# Back to shared code between WKT and GeoJSON

filters = {
    # Your filters here, for example:
    # Cloud cover less than 1%
    "cloud_cover.estimated.lte": 1,
    "intersects": intersects,
    'page.acquired.gt': "2012-01-01T00:00:00"
}

# ortho, landsat, rapideye
url = "https://api.planet.com/v0/scenes/ortho/"

In [ ]:
sf_nw

In [ ]:
import geojson
geojson.


In [ ]:
# send request, get response
r = requests.get(url, params=filters, auth=(myPlanetKey, ''))
r.raise_for_status()
data = r.json()
scenes_data = data["features"]
scene = scenes_data[0]

In [ ]:
len(scenes_data), data['count']

In [ ]:
data['links']

In [ ]:
scene

In [ ]:
# extract fields of interest from scene metadata
timeAcquired = scene["properties"]['acquired']
cloudCoverEst= scene["properties"]['cloud_cover']['estimated']
nrScenes     = data['count']

# links to actual scene data
full_link  = scene["properties"]["links"]['self']
thumb_link = scene["properties"]["links"]['thumbnail']
anal_link  = scene["properties"]['data']['products']['analytic']['full']

# links to next scene
next_url = data["links"].get("next", None) 


In [ ]:
# obtain current scene data

import urllib
import cStringIO 

web_sock   = urllib.urlopen(anal_link)
imgdata    = cStringIO.StringIO(web_sock.read()) # constructs a StringIO holding the image
imgstr     = imgdata.getvalue()

In [ ]:
r = requests.get(anal_link, stream=True, auth=(myPlanetKey, ''))
if 'content-disposition' in r.headers:
    local_filename = r.headers['content-disposition'] \
        .split("filename=")[-1].strip("'\"")
else:
    local_filename = '.'.join(url.split('/')[-2:])
    print local_filename

imgdata = "".join([chunk for chunk in r.iter_content(chunk_size=1024)])

In [ ]:
imgdata==imgstr

In [ ]:
# convert image data to numpy array

import numpy as np
import cv2

nparr = np.fromstring(imgstr, np.uint8)
imgarr = cv2.imdecode(nparr, cv2.IMREAD_LOAD_GDAL)
print imgarr.shape

In [ ]:
imgarr.sum((0,1))

In [ ]:
# save data to disc
with open("tmp.tiff", "wb") as output:
    output.write(imgstr)

# from libtiff import TIFF
# tiff = TIFF.open('tmp2.tiff', mode='w')
# tiff.write_image(imgdata.getvalue())
# tiff.close()

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(imgarr[:,:,1:])

In [ ]:
import requests
import urllib3   

next_url = "https://api.planet.com/v0/scenes/ortho/?" \
    + urllib.urlencode(filters)   

while next_url:
    # Note: you don't have to pass the filters in again here,
    # here, they will always be included in data.links.next
    r = requests.get(next_url, auth=(key, ''))
    r.raise_for_status()
    data = r.json()
    scenes_data = data["features"]
    # do something with scenes_data
    break
    next_url = data["links"].get("next", None) 

# Test own planetlabs API client

In [12]:
import planetlabs as pl

In [13]:
plClient = pl.PlanetLabsAPI(key=myPlanetKey, dataType="scenes", provider="ortho")

In [14]:
scenesData = plClient.get_scenes_metadata(limit=100)

AttributeError: 'module' object has no attribute 'urlencode'

In [ ]:
scenesData[0]

In [ ]:
link = scenesData[0]['properties']['data']['products']['analytic']['full']

img = plClient.get_image_data_from_url(anal_link)

In [ ]:
img.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(img[:,:,1:])

In [ ]:
plt.hist(img[:,:,3].flatten())

In [ ]:
plClient.download_image_from_url(anal_link)

In [ ]:
pl.extract_scene_attributes(scenesData[0], attribute="timeAcquired")

In [ ]:
print locations[0]
scenes = plClient.get_filtered_scenes_metadata(loc=locations[0])